# Intrinsic functions

CoHDL translates function calls by inspecting the abstract syntax tree of the function definition. This is not possible for many Python builtin functions because they

* use language constructs not supported by CoHDL
* are implemented in C and not available as Python code

Some of these functions are treated as CoHDL intrinsics. When intrinsic functions are encountered during compilation they are called with the given arguments and replaced with their return value. Since these functions are never traced by CoHDL they can contain arbitrary code. Intrinsics are a core mechanism for the CoHDL-Python interoperability, features made possible by them in synthesizable contexts include:

* integer operators (`int.__add__`, `int.__gt__`, ...)
* `range`, `zip`, `enumerate`
* `list.__getitem__`, `dict.__getitem__`, `dict.items`
* `len`
* `isinstance`, `issubclass`
* ...


In [1]:
from cohdl import Entity, Port, Unsigned, BitVector
from cohdl import std

class Intrinsics(Entity):
    inp_a = Port.input(BitVector[4])
    inp_b = Port.input(BitVector[4])

    output = Port.output(Unsigned[8])

    def architecture(self):
        @std.concurrent
        def interleave_logic():
            
            # take pairs of input bits and assign them to consecutive output bits
            for nr, (a, b) in enumerate(zip(self.inp_a, self.inp_b)):
                # Since integer addition and multiplication are cohdl builtins
                # they are evaluated at compile time and produce constant indices.
                # The '@' operator is used for concatenation.
                self.output[2*nr+1:2*nr] <<= a @ b


print(std.VhdlCompiler.to_string(Intrinsics))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Intrinsics is
  port (
    inp_a : in std_logic_vector(3 downto 0);
    inp_b : in std_logic_vector(3 downto 0);
    output : out unsigned(7 downto 0)
    );
end Intrinsics;


architecture arch_Intrinsics of Intrinsics is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_output : unsigned(7 downto 0);
  signal temp : std_logic_vector(1 downto 0);
  signal temp_1 : std_logic_vector(1 downto 0);
  signal temp_2 : std_logic_vector(1 downto 0);
  signal temp_3 : std_logic_vector(1 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  output <= buffer_output;
  
  -- CONCURRENT BLOCK (interleave_logic)
  temp <= (inp_a(0)) & (inp_b(0));
  buffer_output(1 downto 0) <= unsigned(temp);
  temp_1 <= (inp_a(1)) & (inp_b(1));
  buffer_output(3 downto 2) <=

## consteval decorator

The consteval decorator (named after the C++20 consteval keyword) turns user defined functions into CoHDL intrinsics. The decorator returns the function object unchanged. Internally it writes the functions id into a set of all intrinsics. The compiler checks that list before attempting to inspect the functions AST.

In [2]:
from cohdl import Entity, Port, Bit, Unsigned, consteval
from cohdl import std

@consteval
def load_from_file(param_name):
    # arbitrary python code
    # ...

    # dummy value used for example
    return 0xAAAA

class ConstevalExample(Entity):
    clk = Port.input(Bit)

    output = Port.output(Unsigned[16])

    def architecture(self):
        clk = std.Clock(self.clk)

        @std.sequential(clk)
        def logic():
            self.output <<= load_from_file("my_param_name")

print(std.VhdlCompiler.to_string(ConstevalExample))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity ConstevalExample is
  port (
    clk : in std_logic;
    output : out unsigned(15 downto 0)
    );
end ConstevalExample;


architecture arch_ConstevalExample of ConstevalExample is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_output : unsigned(15 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  output <= buffer_output;
  

  logic: process(clk)
  begin
    if rising_edge(clk) then
      buffer_output <= unsigned'("1010101010101010");
    end if;
  end process;
end architecture arch_ConstevalExample;
